https://www.youtube.com/watch?v=OIenNRt2bjg intro to pytorch.

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils
from tqdm import tqdm
from scipy.special import erf
import scipy.stats

from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from dmipy.core.modeling_framework import MultiCompartmentSphericalMeanModel
from dmipy.signal_models import sphere_models, cylinder_models, gaussian_models

from scipy.io import savemat

/Users/admin/opt/anaconda3/envs/project/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Defining the verdict model compartments. 

- gradient_strength = np.array([np.sqrt(b_values[i])/(gamma*delta[i]*np.sqrt(Delta[i]-delta[i]/3)) for i,_ in enumerate(b_values)])
    -   What does the underscore mean? It might mean an usused variable, but there's only 1 dimension to b_values
    - If I did for in enumerate(b_values) then you get back a index and a value, but the i,_ returns just the index and ignores the value. It is a blank unused variable. 

- What is the purpose of the error function?
    - First look at where the astrosticks formula is from, then ask again.

In [ ]:

# simulate data according to verdict model - there is a function for each of the three compartments
# when you do it for bbb-fexi, this will change

def sphere(r):
    """ Returns 1x10 array
    """
    # Create 1d array with these 100 values
    SPHERE_TRASCENDENTAL_ROOTS = np.r_[
        # 0.,
        2.081575978, 5.940369990, 9.205840145,
        12.40444502, 15.57923641, 18.74264558, 21.89969648,
        25.05282528, 28.20336100, 31.35209173, 34.49951492,
        37.64596032, 40.79165523, 43.93676147, 47.08139741,
        50.22565165, 53.36959180, 56.51327045, 59.65672900,
        62.80000055, 65.94311190, 69.08608495, 72.22893775,
        75.37168540, 78.51434055, 81.65691380, 84.79941440,
        87.94185005, 91.08422750, 94.22655255, 97.36883035,
        100.5110653, 103.6532613, 106.7954217, 109.9375497,
        113.0796480, 116.2217188, 119.3637645, 122.5057870,
        125.6477880, 128.7897690, 131.9317315, 135.0736768,
        138.2156061, 141.3575204, 144.4994207, 147.6413080,
        150.7831829, 153.9250463, 157.0668989, 160.2087413,
        163.3505741, 166.4923978, 169.6342129, 172.7760200,
        175.9178194, 179.0596116, 182.2013968, 185.3431756,
        188.4849481, 191.6267147, 194.7684757, 197.9102314,
        201.0519820, 204.1937277, 207.3354688, 210.4772054,
        213.6189378, 216.7606662, 219.9023907, 223.0441114,
        226.1858287, 229.3275425, 232.4692530, 235.6109603,
        238.7526647, 241.8943662, 245.0360648, 248.1777608,
        251.3194542, 254.4611451, 257.6028336, 260.7445198,
        263.8862038, 267.0278856, 270.1695654, 273.3112431,
        276.4529189, 279.5945929, 282.7362650, 285.8779354,
        289.0196041, 292.1612712, 295.3029367, 298.4446006,
        301.5862631, 304.7279241, 307.8695837, 311.0112420,
        314.1528990
    ]

    D = 2
    gamma = 2.67e2
    radius = r

    b_values = np.array([1e-6, 0.090, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
    Delta = np.array([23.8, 23.8, 23.8, 31.3, 23.8, 43.8, 23.8, 34.3, 23.8, 38.8])
    delta = np.array([3.9, 3.9, 3.9, 11.4, 3.9, 23.9, 3.9, 14.4, 3.9, 18.9])

    #calculate gradstrength for each b value and (D/d)elta. i is an index. 
    gradient_strength = np.array([np.sqrt(b_values[i])/(gamma*delta[i]*np.sqrt(Delta[i]-delta[i]/3)) for i,_ in enumerate(b_values)])

    # alpha, alpha2 and alpha2d are a 100x1 arrays, the maths is elementwise. 
    alpha = SPHERE_TRASCENDENTAL_ROOTS / radius
    #** means squared, also done elementwise
    alpha2 = alpha ** 2
    alpha2D = alpha2 * D

    #create an array of 1x10. Seems to always be the same. 
    first_factor = -2 * (gamma * gradient_strength) ** 2 / D
    
    #create zeros array with lengths as sizes. In this case 100x10
    summands = np.zeros((len(SPHERE_TRASCENDENTAL_ROOTS),len(b_values)))
    
    #create 100x10 array called summands
    #equation is summation on page 5 of ssVERDICT paper.
    for i,_ in enumerate(delta):
        summands[:,i] = (
            alpha ** (-4) / (alpha2 * radius ** 2 - 2) *
            (
                2 * delta[i] - (
                    2 +
                    np.exp(-alpha2D * (Delta[i] - delta[i])) -
                    2 * np.exp(-alpha2D * delta[i]) -
                    2 * np.exp(-alpha2D * Delta[i]) +
                    np.exp(-alpha2D * (Delta[i] + delta[i]))
                ) / (alpha2D)
            )
        )
    #sum all 100 elements in summands, and create E, an array which is 10x1
    E = np.exp(
        first_factor *
        summands.sum()
    )
    return E

def ball(d):
    """ Returns 1x10 array
    """
    bvals = np.array([1e-6, 0.090, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
    E_ball = np.exp(-bvals * d)
    
    #E_ball is a 1x10 array
    return E_ball

def astrosticks(l):
    """ Returns 1x10 array
    """
    bvals = np.array([1e-6, 0.090, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
    lambda_par = l

    #make an array filled with ones, same size as bvals (1x10)
    E_mean = np.ones_like(bvals)
    #erf() is the error function, returns values between -1 to 1. I think it is like normalising a value perhaps.
    E_mean = ((np.sqrt(np.pi) * erf(np.sqrt(bvals * lambda_par))) /
                (2 * np.sqrt(bvals * lambda_par)))

    #E_mean is a 1x10 array
    return E_mean 


### Defining number of voxels and verdict parameters
- Why is 10 an important and recurring number? Is that the number of b values used?
- What does expand_dims() do? It seems to add an extra dimension, but why is that needed?
- fvasc seems to be a fraction, not an absolute value. Is this different to fic and fees and why?
    - They all eventually become fractions, a few lines later on. 

In [ ]:

nvox = 1000 # number of voxels to simulate

#These 3 are all 1x nvox arrays. 
radii = np.random.uniform(0.001,15,nvox) # free parameter - cell radius
dees = np.random.uniform(0.5,3,nvox) # free parameter - EES diffusivity
lambdapar = np.repeat(2,nvox) # fixed parameter

# These are all nvox x 10 arrays. (10 is number of bvals, so I think acquisitions.)
E_stick = np.array([astrosticks(l) for l in lambdapar]) 
E_ball = np.array([ball(d) for d in dees])
E_sphere = np.array([sphere(r) for r in radii])

# nvox x 1 arrays. 
fic = np.expand_dims(np.random.uniform(0.001, 0.999, nvox), axis=1) # free parameter - IC volume fraction
fees = np.expand_dims(np.random.uniform(0.001, 0.999, nvox), axis=1) # free parameter - EES volume fraction
fvasc = 1 - fic - fees # calculate VASC volume fraction
fvasc = fvasc/(fic + fees + fvasc)

A = fvasc
#Smallest value now becomes the new zero, similar to kelvin and celsius. 
#Done for all elements, so normA is nvox x 1 array
normA = A - min(A)
fvasc = 0.2 * (normA/max(normA)) # constraining fvasc to be realistic for prostate tissue (under 0.2)

#changing into fractions. 
fic = fic/(fic + fees + fvasc)
fees = fees/(fic + fees + fvasc)

# These are all nvox x 10 arrays. Adding noise
# E_vox is the signal
E_vox = fees*E_ball + fic*E_sphere + fvasc*E_stick
E_vox_real = E_vox + np.random.normal(scale=0.02, size=np.shape(E_vox)) # adding rician noise, snr = 50
E_vox_imag = np.random.normal(scale=0.02, size=np.shape(E_vox))
E_vox = np.sqrt(E_vox_real**2 + E_vox_imag**2) # these are the simulated signals


### Section for comparing to NLLS fitting.
Don't know what it does yet.

In [ ]:


'''

## this section will be useful when you want to compare it to NLLS fitting, i'm commenting it out for now

b_values = np.array([1e-6, 90, 1e-6, 500, 1e-6, 1500, 1e-6, 2000, 1e-6, 3000])
bvaluesSI = np.array([i * 1e6 for i in b_values])
Delta = np.array([0.0238, 0.0238, 0.0238, 0.0313, 0.0238, 0.0438, 0.0238, 0.0343, 0.0238, 0.0388])
delta = np.array([0.0039, 0.0039, 0.0039, 0.0114, 0.0039, 0.0239, 0.0039, 0.0144, 0.0039, 0.0189])
gradient_directions = np.loadtxt('./verdict_graddirs.txt', delimiter=',')
acq_scheme = acquisition_scheme_from_bvalues(bvaluesSI, gradient_directions, delta, Delta)

spheresim = sphere_models.S4SphereGaussianPhaseApproximation(diffusion_constant=2e-9)
ballsim = gaussian_models.G1Ball()
sticksim = cylinder_models.C1Stick(lambda_par=2e-9)
astro = sticksim.spherical_mean(acq_scheme)

from dmipy.core.modeling_framework import MultiCompartmentModel
verdict_mod = MultiCompartmentModel(models=[spheresim, ballsim, sticksim])
verdict_mod.set_parameter_optimization_bounds('G1Ball_1_lambda_iso', [0.5e-9, 3e-9])
verdict_mod.set_parameter_optimization_bounds('partial_volume_0', [0.001, 0.999])
verdict_mod.set_parameter_optimization_bounds('partial_volume_1', [0.001, 0.999])
verdict_mod.set_parameter_optimization_bounds('partial_volume_1', [0.001, 0.199])
verdict_mod.set_parameter_optimization_bounds('G1Ball_1_lambda_iso', [0.5e-9, 3e-9])


res = verdict_mod.fit(acq_scheme, E_vox)
f_ic = res.fitted_parameters['partial_volume_0']
f_ees = res.fitted_parameters['partial_volume_1']
f_vasc = res.fitted_parameters['partial_volume_2']
r = res.fitted_parameters['S4SphereGaussianPhaseApproximation_1_diameter']/2
d_ees = res.fitted_parameters['G1Ball_1_lambda_iso']
'''


### Creating the neural network
+ What does self.b_values = b_values do?
    + assignes b values to the "self" version  of b values, ie for that time it was called.
+ self.encoder = nn.Sequential(*self.layers, nn.Linear(len(b_values), nparams))
    What does this line do? Seems inportant
+ Should this be D instead of 2? It is in the first box, so why is this different?
    + alpha2D = alpha2 * 2
    + NameError: name 'D' is not defined is the error if I put D in. 
+ What is going on with all the unsqeezes?
+ What makes summands be 100 long? 
    + I get the other 2 dimensions
+ What is X?!

In [ ]:

class Net(nn.Module): # this is the neural network

    #defining the init and foward pass functions. 

    def __init__(self, b_values, Delta, delta, gradient_strength, nparams):
        super(Net, self).__init__()

        self.b_values = b_values
        self.Delta = Delta
        self.delta = delta
        self.gradient_strength = gradient_strength


        #defining the layers that we want. 
        # 3 layers with 10 (no. of b_values) nodes. See paper.
        self.layers = nn.ModuleList()
        for i in range(3): # 3 fully connected hidden layers
            self.layers.extend([nn.Linear(len(b_values), len(b_values)), nn.PReLU()])
            #https://pytorch.org/docs/stable/generated/torch.nn.PReLU.html
        self.encoder = nn.Sequential(*self.layers, nn.Linear(len(b_values), nparams))

    def forward(self, X):
        """
        constrain f_ic, f_ees, r, d_ees and calculate x.
        """
        params = torch.nn.functional.softplus(self.encoder(X))
        #SoftPlus is a smooth approximation to the ReLU function and can be used to constrain the output of a machine to always be positive.


        #params seems be a table containing each variable in a new column. 
        #unsqueeze adds an additional dimension. 
        f_ic = torch.clamp(params[:,0].unsqueeze(1), min=0.001, max=0.999) # parameter constraints
        f_ees = torch.clamp(params[:,1].unsqueeze(1), min=0.001, max=0.999)
        r = torch.clamp(params[:,2].unsqueeze(1), min=0.001, max=14.999)
        d_ees = torch.clamp(params[:,3].unsqueeze(1), min=0.5, max=3)

        # Create 1d array with these 100 values
        SPHERE_TRASCENDENTAL_ROOTS = np.r_[
        # 0.,
        2.081575978, 5.940369990, 9.205840145,
        12.40444502, 15.57923641, 18.74264558, 21.89969648,
        25.05282528, 28.20336100, 31.35209173, 34.49951492,
        37.64596032, 40.79165523, 43.93676147, 47.08139741,
        50.22565165, 53.36959180, 56.51327045, 59.65672900,
        62.80000055, 65.94311190, 69.08608495, 72.22893775,
        75.37168540, 78.51434055, 81.65691380, 84.79941440,
        87.94185005, 91.08422750, 94.22655255, 97.36883035,
        100.5110653, 103.6532613, 106.7954217, 109.9375497,
        113.0796480, 116.2217188, 119.3637645, 122.5057870,
        125.6477880, 128.7897690, 131.9317315, 135.0736768,
        138.2156061, 141.3575204, 144.4994207, 147.6413080,
        150.7831829, 153.9250463, 157.0668989, 160.2087413,
        163.3505741, 166.4923978, 169.6342129, 172.7760200,
        175.9178194, 179.0596116, 182.2013968, 185.3431756,
        188.4849481, 191.6267147, 194.7684757, 197.9102314,
        201.0519820, 204.1937277, 207.3354688, 210.4772054,
        213.6189378, 216.7606662, 219.9023907, 223.0441114,
        226.1858287, 229.3275425, 232.4692530, 235.6109603,
        238.7526647, 241.8943662, 245.0360648, 248.1777608,
        251.3194542, 254.4611451, 257.6028336, 260.7445198,
        263.8862038, 267.0278856, 270.1695654, 273.3112431,
        276.4529189, 279.5945929, 282.7362650, 285.8779354,
        289.0196041, 292.1612712, 295.3029367, 298.4446006,
        301.5862631, 304.7279241, 307.8695837, 311.0112420,
        314.1528990
        ]
        
        # all are 100x1 arrays
        alpha = torch.FloatTensor(SPHERE_TRASCENDENTAL_ROOTS) / (r)
        alpha2 = alpha ** 2
        alpha2D = alpha2 * 2

        #add extra dimension
        alpha = alpha.unsqueeze(1)
        alpha2 = alpha2.unsqueeze(1)
        alpha2D = alpha2D.unsqueeze(1)

        gamma = 2.675987e2
        #elsewhere gradient_strength is 10x1, so I think first factor is also 10x1.
        first_factor = -2*(gamma*self.gradient_strength)**2 / 2

        #adds 2 dimensions. 
        delta = self.delta.unsqueeze(0).unsqueeze(2)
        Delta = self.Delta.unsqueeze(0).unsqueeze(2)
        
        #Size of summands is [128, 10, 100]
        #128 is batch size, 10 is b_values, not sure what 100 roots.
        #create 100x10 array called summands
        #equation is summation on page 5 of ssVERDICT paper.
        summands = (alpha ** (-4) / (alpha2 * (r.unsqueeze(2))**2 - 2) * (
                            2 * delta - (
                            2 +
                            torch.exp(-alpha2D * (Delta - delta)) -
                            2 * torch.exp(-alpha2D * delta) -
                            2 * torch.exp(-alpha2D * Delta) +
                            torch.exp(-alpha2D * (Delta + delta))
                        ) / (alpha2D)
                    )
                )
         
        # this is where you will eventually need to tweak the network to be bbb-fexi rather than verdict

        #erf() is error function.
        #xi,xii,xiii and x are all 128x10. So batch size x 10 b_values most likely.
        xi = (1 - f_ic - f_ees) * ((np.sqrt(np.pi) * torch.erf(np.sqrt(self.b_values * 2))) /
                (2 * np.sqrt(self.b_values * 2)))
        xii = f_ic * torch.exp(torch.FloatTensor(first_factor) * torch.sum(summands, 2))
        xiii = f_ees * torch.exp(-self.b_values * d_ees)       
        X = xi + xii + xiii

        return X, f_ic, f_ees, r, d_ees



### NN continued

In [ ]:
# define network
nparams = 4
b_values = torch.FloatTensor([1e-6, 0.090, 1e-6, 0.500, 1e-6, 1.5, 1e-6, 2, 1e-6, 3])
Delta = torch.FloatTensor([23.8, 23.8, 23.8, 31.3, 23.8, 43.8, 23.8, 34.3, 23.8, 38.8])
delta = torch.FloatTensor([3.9, 3.9, 3.9, 11.4, 3.9, 23.9, 3.9, 14.4, 3.9, 18.9])
gamma = 2.67e2
#gradient_strength is 10x1.
gradient_strength = torch.FloatTensor([np.sqrt(b_values[i])/(gamma*delta[i]*np.sqrt(Delta[i]-delta[i]/3)) for i,_ in enumerate(b_values)])

#initilise network
net = Net(b_values, Delta, delta, gradient_strength, nparams)

#create batch queues for data
batch_size = 128
#// means divide and round down. 
num_batches = len(E_vox) // batch_size

#import the E_vox array into the dataloader amd convert to float.
#drop_last ignores the last batch if it is the wrong size. 
#num_workers is about performance. 
trainloader = utils.DataLoader(torch.from_numpy(E_vox.astype(np.float32)),
                                batch_size = batch_size, 
                                shuffle = True,
                                num_workers = 0, #was 2 previously
                                drop_last = True)

# loss function and optimizer
#choosing which loss function to use. 
#not sure what the optmizer is
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0001)

# best loss
best = 1e16
num_bad_epochs = 0
#can increase patients a lot, speed not an issue.
patience = 10



### Training
+ what does net.train() do?

In [ ]:

# train
for epoch in range(10000): 
    print("-----------------------------------------------------------------")
    print("epoch: {}; bad epochs: {}".format(epoch, num_bad_epochs))
    net.train()
    running_loss = 0.

    #tqdm shows a progress bar. 
    for i, X_batch in enumerate(tqdm(trainloader), 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        X_pred, f_ic_pred, f_ees_pred, r_pred, d_ees_pred = net(X_batch)
        loss = criterion(X_pred, X_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
      
    print("loss: {}".format(running_loss))
    # early stopping
    if running_loss < best:
        print("####################### saving good model #######################")
        final_model = net.state_dict()
        best = running_loss
        num_bad_epochs = 0
    else:
        num_bad_epochs = num_bad_epochs + 1
        if num_bad_epochs == patience:
            print("done, best loss: {}".format(best))
            break
print("done")

net.load_state_dict(final_model)

net.eval()
with torch.no_grad():
    X_real_pred, f_ic, f_ees, r, d_ees = net(torch.from_numpy(E_vox.astype(np.float32)))

f_vasc = 1 - f_ic - f_ees

f_vasc = f_vasc/(f_ic + f_ees + f_vasc)
A = f_vasc
normA = A - min(A)
f_vasc = 0.2 * (normA/max(normA))
f_ic = f_ic/(f_ic + f_ees + f_vasc)
f_ees = f_ees/(f_ic + f_ees + f_vasc)


### Plots

In [ ]:
# check predicted signal against simulated signal

plt.scatter(b_values, E_vox[0,:], label='simulated')
plt.scatter(b_values, X_real_pred[0,:], label='predicted')
plt.legend()

# plot scatter plots to analyse correlation of predicted free params against ground truth
plt.figure()

param = [fic, fees, fvasc, radii, dees]
param_f = [f_ic, f_ees, f_vasc, r, d_ees]
param_name = ['fIC', 'fEES', 'fVASC', 'R', 'dEES']
rvals = []

for i,_ in enumerate(param):
    plt.rcParams['font.size'] = '16'
    plt.scatter(param[i], param_f[i], s=2, c='navy')
    plt.xlabel(param_name[i] + ' Ground Truth')
    plt.ylabel(param_name[i] + ' Prediction')
    rvals.append(scipy.stats.pearsonr(np.squeeze(param[i]), np.squeeze(param_f[i])))
    plt.tight_layout
    plt.show()

print(rvals)



### Bias-variance calculations
+ this section crashes when using 100,000 voxels, although my laptop did switch off. 

In [ ]:
#split up and see where it breaks. try convert to numpy and use other section.
## bias-variance calculations

bias_fic = torch.mean(f_ic - fic)
bias_fees = torch.mean(f_ees - fees)
bias_r = torch.mean(r - radii)
bias_dees = torch.mean(d_ees - dees)

var_fic = torch.mean((f_ic - torch.mean(f_ic))**2)
var_fees = torch.mean((f_ees - torch.mean(f_ees))**2)
var_r = torch.mean((r - torch.mean(r))**2)
var_dees = torch.mean((d_ees - torch.mean(d_ees))**2)

mse_fic = torch.mean((f_ic - fic)**2)
mse_fees = torch.mean((f_ees - fees)**2)
mse_r = torch.mean((r - radii)**2)
mse_dees = torch.mean((d_ees - dees)**2)

print(bias_fic, bias_fees, bias_r, bias_dees)
print(var_fic, var_fees, var_r, var_dees)
print(mse_fic, mse_fees, mse_r, mse_dees)


r = r*1e6
d_ees = d_ees*1e9


### Bias-variance calculations using numpy

In [ ]:

'''
bias_fic = np.mean(f_ic - fic)
bias_fees = np.mean(f_ees - fees)
bias_r = np.mean(r - radii)
bias_dees = np.mean(d_ees - dees)

var_fic = np.mean((f_ic - np.mean(f_ic))**2)
var_fees = np.mean((f_ees - np.mean(f_ees))**2)
var_r = np.mean((r - np.mean(r))**2)
var_dees = np.mean((d_ees - np.mean(d_ees))**2)

mse_fic = np.mean((f_ic - fic)**2)
mse_fees = np.mean((f_ees - fees)**2)
mse_r = np.mean((r - radii)**2)
mse_dees = np.mean((d_ees - dees)**2)

print(bias_fic, bias_fees, bias_r, bias_dees)
print(var_fic, var_fees, var_r, var_dees)
print(mse_fic, mse_fees, mse_r, mse_dees)'''